## 来自

https://zhuanlan.zhihu.com/p/26177617

## 摘要
### 如何构造训练集/测试集
> 1. 列别特征one-hot编码

> 2. 用户和商品的累计特征

> 3. 滑动窗口特征／时间衰减

> 4. 训练集／测试集划分

### xgboost模型


### 评测代码

In [78]:
import time
from datetime import datetime
from datetime import timedelta
import pandas as pd
import pickle
import os
import math
import numpy as np

In [4]:
action_1_path = "./data/JData_Action_201602.csv"
action_2_path = "./data/JData_Action_201603.csv"
action_3_path = "./data/JData_Action_201604.csv"
comment_path = "./data/JData_Comment.csv"
product_path = "./data/JData_Product.csv"
user_path = "./data/JData_User.csv"

comment_date = ["2016-02-01", "2016-02-08", "2016-02-15", "2016-02-22", "2016-02-29", "2016-03-07", "2016-03-14",
                "2016-03-21", "2016-03-28", "2016-04-04", "2016-04-11", "2016-04-15"]


def convert_age(age_str):
    if age_str == u'-1':
        return 0
    elif age_str == u'15岁以下':
        return 1
    elif age_str == u'16-25岁':
        return 2
    elif age_str == u'26-35岁':
        return 3
    elif age_str == u'36-45岁':
        return 4
    elif age_str == u'46-55岁':
        return 5
    elif age_str == u'56岁以上':
        return 6
    else:
        return -1

    
def get_actions_1():
    action = pd.read_csv(action_1_path)
    return action

def get_actions_2():
    action2 = pd.read_csv(action_2_path)
    return action2

def get_actions_3():
    action3 = pd.read_csv(action_3_path)
    return action3


def get_actions(start_date, end_date):
    """

    :param start_date:
    :param end_date:
    :return: actions: pd.Dataframe
    """
    dump_path = './cache/all_action_%s_%s.pkl' % (start_date, end_date)
    if os.path.exists(dump_path):
        actions = pickle.load(open(dump_path))
    else:
        action_1 = get_actions_1()
        action_2 = get_actions_2()
        action_3 = get_actions_3()
        actions = pd.concat([action_1, action_2, action_3]) # type: pd.DataFrame
        actions = actions[(actions.time >= start_date) & (actions.time < end_date)]
        pickle.dump(actions, open(dump_path, 'w'))
    return actions


In [5]:
# 用户特征使用了age，sex，lv_cd
def get_basic_user_feat():
    dump_path = './cache/basic_user.pkl'
    if os.path.exists(dump_path):
        user = pickle.load(open(dump_path))
    else:
        user = pd.read_csv(user_path, encoding='gbk')
        user['age'] = user['age'].map(convert_age)
        age_df = pd.get_dummies(user["age"], prefix="age")
        sex_df = pd.get_dummies(user["sex"], prefix="sex")
        user_lv_df = pd.get_dummies(user["user_lv_cd"], prefix="user_lv_cd")
        user = pd.concat([user['user_id'], age_df, sex_df, user_lv_df], axis=1)
        pickle.dump(user, open(dump_path, 'w'))
    return user

In [6]:
get_basic_user_feat()

,user_id,age_0,age_1,age_2,age_3,age_4,age_5,age_6,sex_0,sex_1,sex_2,user_lv_cd_1,user_lv_cd_2,user_lv_cd_3,user_lv_cd_4,user_lv_cd_5
0,54,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0
1,79,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0
2,100,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0
3,266,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0
4,333,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
5,605,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0
6,638,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1
7,686,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0
8,692,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0
9,819,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0


In [7]:
# 商品维度
def get_basic_product_feat():
    dump_path = './cache/basic_product.pkl'
    if os.path.exists(dump_path):
        product = pickle.load(open(dump_path))
    else:
        product = pd.read_csv(product_path)
        attr1_df = pd.get_dummies(product["attr1"], prefix="a1")
        attr2_df = pd.get_dummies(product["attr2"], prefix="a2")
        attr3_df = pd.get_dummies(product["attr3"], prefix="a3")
        product = pd.concat([product[['sku_id', 'cate', 'brand']], attr1_df, attr2_df, attr3_df], axis=1)
        pickle.dump(product, open(dump_path, 'w'))
    return product

In [8]:
get_basic_product_feat()

,sku_id,cate,brand,a1_-1,a1_1,a1_2,a1_3,a2_-1,a2_1,a2_2,a3_-1,a3_1,a3_2
0,166731,8,545,0,1,0,0,1,0,0,1,0,0
1,146488,8,812,0,0,1,0,0,0,1,1,0,0
2,70546,8,214,0,0,0,1,0,1,0,0,1,0
3,92487,8,812,0,0,1,0,0,0,1,1,0,0
4,113065,8,812,0,0,1,0,0,0,1,1,0,0
5,80307,8,812,0,0,1,0,0,0,1,1,0,0
6,71669,8,812,0,0,1,0,0,1,0,1,0,0
7,4145,8,812,0,0,1,0,0,0,1,0,1,0
8,35450,8,812,0,0,1,0,0,0,1,1,0,0
9,131403,8,812,0,0,1,0,0,1,0,1,0,0


In [9]:
# 用户累积特征
def get_accumulate_user_feat(start_date, end_date):
    feature = ['user_id', 'user_action_1_ratio', 'user_action_2_ratio', 'user_action_3_ratio',
               'user_action_5_ratio', 'user_action_6_ratio']
    dump_path = './cache/user_feat_accumulate_%s_%s.pkl' % (start_date, end_date)
    if os.path.exists(dump_path):
        actions = pickle.load(open(dump_path))
    else:
        actions = get_actions(start_date, end_date)
        df = pd.get_dummies(actions['type'], prefix='action')
        actions = pd.concat([actions['user_id'], df], axis=1)
        actions = actions.groupby(['user_id'], as_index=False).sum()
        actions['user_action_1_ratio'] = actions['action_4'] / actions['action_1']
        actions['user_action_2_ratio'] = actions['action_4'] / actions['action_2']
        actions['user_action_3_ratio'] = actions['action_4'] / actions['action_3']
        actions['user_action_5_ratio'] = actions['action_4'] / actions['action_5']
        actions['user_action_6_ratio'] = actions['action_4'] / actions['action_6']
        actions = actions[feature]
        pickle.dump(actions, open(dump_path, 'w'))
    return actions

In [10]:
start_days = "2016-02-01"
train_end_date = '2016-04-11'

user_acc = get_accumulate_user_feat(start_days, train_end_date)
user_acc

,user_id,user_action_1_ratio,user_action_2_ratio,user_action_3_ratio,user_action_5_ratio,user_action_6_ratio
0,1,0.000000,NaN,NaN,NaN,0.000000
1,2,0.000000,NaN,NaN,NaN,0.000000
2,3,0.066667,2.000000,2.000000,inf,0.029412
3,4,0.000000,NaN,NaN,NaN,0.000000
4,5,0.000000,NaN,NaN,NaN,0.000000
5,6,0.000000,NaN,NaN,NaN,0.000000
6,7,0.020833,0.166667,0.500000,inf,0.007937
7,8,0.000000,0.000000,NaN,NaN,0.000000
8,9,0.022727,0.500000,1.000000,inf,0.017241
9,10,0.000000,NaN,NaN,NaN,0.000000


In [12]:
# 商品下单比例与浏览，加入购物车，购物车删除，关注，点击比例
# 商品累积特征
def get_accumulate_product_feat(start_date, end_date):
    feature = ['sku_id', 'product_action_1_ratio', 'product_action_2_ratio', 'product_action_3_ratio',
               'product_action_5_ratio', 'product_action_6_ratio']
    dump_path = './cache/product_feat_accumulate_%s_%s.pkl' % (start_date, end_date)
    if os.path.exists(dump_path):
        actions = pickle.load(open(dump_path))
    else:
        actions = get_actions(start_date, end_date)
        df = pd.get_dummies(actions['type'], prefix='action')
        actions = pd.concat([actions['sku_id'], df], axis=1)
        actions = actions.groupby(['sku_id'], as_index=False).sum()
        actions['product_action_1_ratio'] = actions['action_4'] / actions['action_1']
        actions['product_action_2_ratio'] = actions['action_4'] / actions['action_2']
        actions['product_action_3_ratio'] = actions['action_4'] / actions['action_3']
        actions['product_action_5_ratio'] = actions['action_4'] / actions['action_5']
        actions['product_action_6_ratio'] = actions['action_4'] / actions['action_6']
        actions = actions[feature]
        pickle.dump(actions, open(dump_path, 'w'))
    return actions

In [13]:
start_days = "2016-02-01"
train_end_date = '2016-04-11'
product_acc = get_accumulate_product_feat(start_days, train_end_date)
product_acc

,sku_id,product_action_1_ratio,product_action_2_ratio,product_action_3_ratio,product_action_5_ratio,product_action_6_ratio
0,2,0.000000,NaN,NaN,NaN,0.000000
1,18,0.000000,NaN,NaN,NaN,0.000000
2,23,0.000000,NaN,NaN,NaN,0.000000
3,36,0.000000,NaN,0.000000,0.000000,0.000000
4,37,0.000000,NaN,NaN,NaN,0.000000
5,40,0.000000,0.000000,0.000000,NaN,0.000000
6,41,0.000000,NaN,NaN,NaN,NaN
7,43,0.000000,0.000000,0.000000,0.000000,0.000000
8,44,0.000000,NaN,NaN,NaN,0.000000
9,45,0.000000,NaN,NaN,NaN,0.000000


In [21]:
# 商品评论
def get_comments_product_feat(start_date, end_date):
    dump_path = './cache/comments_accumulate_%s_%s.pkl' % (start_date, end_date)
    if os.path.exists(dump_path):
        comments = pickle.load(open(dump_path))
    else:
        comments = pd.read_csv(comment_path)
        comment_date_end = end_date
        comment_date_begin = comment_date[0]
        for date in reversed(comment_date):
            if date < comment_date_end:
                comment_date_begin = date
                print "comment_date_begin: ", comment_date_begin
                break
        comments = comments[(comments.dt >= comment_date_begin) & (comments.dt < comment_date_end)]
        df = pd.get_dummies(comments['comment_num'], prefix='comment_num')
        comments = pd.concat([comments, df], axis=1) # type: pd.DataFrame
        #del comments['dt']
        #del comments['comment_num']
        comments = comments[['sku_id', 'has_bad_comment', 'bad_comment_rate', 'comment_num_1', 'comment_num_2', 'comment_num_3', 'comment_num_4']]
        pickle.dump(comments, open(dump_path, 'w'))
    return comments


In [23]:
start_days = "2016-02-01"
train_end_date = '2016-04-11'

comment_acc = get_comments_product_feat(start_days, train_end_date)
comment_acc

comment_date_begin:  2016-04-04


,sku_id,has_bad_comment,bad_comment_rate,comment_num_1,comment_num_2,comment_num_3,comment_num_4
325822,115070,0,0.0000,0,1,0,0
325823,117428,0,0.0000,0,1,0,0
325824,54663,0,0.0000,0,1,0,0
325825,9154,0,0.0000,1,0,0,0
325826,125794,0,0.0000,0,1,0,0
325827,107320,0,0.0000,1,0,0,0
325828,34105,0,0.0000,1,0,0,0
325829,60922,0,0.0000,0,0,0,1
325830,149389,0,0.0000,0,0,1,0
325831,23835,0,0.0000,0,1,0,0


In [24]:
def get_labels(start_date, end_date):
    dump_path = './cache/labels_%s_%s.pkl' % (start_date, end_date)
    if os.path.exists(dump_path):
        actions = pickle.load(open(dump_path))
    else:
        actions = get_actions(start_date, end_date)
        actions = actions[actions['type'] == 4]
        actions = actions.groupby(['user_id', 'sku_id'], as_index=False).sum()
        actions['label'] = 1
        actions = actions[['user_id', 'sku_id', 'label']]
        pickle.dump(actions, open(dump_path, 'w'))
    return actions

In [25]:
test_start_date = '2016-04-11'
test_end_date = '2016-04-16'
labels = get_labels(test_start_date, test_end_date)
labels

,user_id,sku_id,label
0,100,101412,1
1,128,57161,1
2,137,131300,1
3,152,28276,1
4,160,51207,1
5,172,135376,1
6,220,84389,1
7,235,169085,1
8,247,28276,1
9,271,80497,1


In [37]:
# 滑动窗口特征
def get_action_feat(start_date, end_date):
    dump_path = './cache/action_accumulate_%s_%s.pkl' % (start_date, end_date)
    if os.path.exists(dump_path):
        actions = pickle.load(open(dump_path))
    else:
        actions = get_actions(start_date, end_date)
        actions = actions[['user_id', 'sku_id', 'type']]
        df = pd.get_dummies(actions['type'], prefix='%s-%s-action' % (start_date, end_date))
        actions = pd.concat([actions, df], axis=1)  # type: pd.DataFrame
        actions = actions.groupby(['user_id', 'sku_id'], as_index=False).sum()
        del actions['type']
        pickle.dump(actions, open(dump_path, 'w'))
    return actions

# make train set
def make_train_set(train_start_date, train_end_date, test_start_date, test_end_date):
    dump_path = './cache/train_set_%s_%s_%s_%s.pkl' % (train_start_date, train_end_date, test_start_date, test_end_date)
    if os.path.exists(dump_path):
        actions = pickle.load(open(dump_path))
    else:
        start_days = "2016-02-01"
        start_days = "2016-02-01"
        user = get_basic_user_feat()
        product = get_basic_product_feat()
        user_acc = get_accumulate_user_feat(start_days, train_end_date)
        product_acc = get_accumulate_product_feat(start_days, train_end_date)
        comment_acc = get_comments_product_feat(train_start_date, train_end_date)
        labels = get_labels(test_start_date, test_end_date)
        
        actions = None
        for i in (1, 2, 3, 5, 7, 10, 15, 21, 30):
            start_days = datetime.strptime(train_end_date, '%Y-%m-%d') - timedelta(days=i)
            start_days = start_days.strftime('%Y-%m-%d')
            if actions is None:
                actions = get_action_feat(start_days, train_end_date)
            else:
                actions = pd.merge(actions, get_action_feat(start_days, train_end_date), how='left',
                                   on=['user_id', 'sku_id'])

        actions = pd.merge(actions, user, how='left', on='user_id')
        actions = pd.merge(actions, user_acc, how='left', on='user_id')
        actions = pd.merge(actions, product, how='left', on='sku_id')
        actions = pd.merge(actions, product_acc, how='left', on='sku_id')
        actions = pd.merge(actions, comment_acc, how='left', on='sku_id')
        actions = pd.merge(actions, labels, how='left', on=['user_id', 'sku_id'])
        actions = actions.fillna(0)
        pickle.dump(actions, open(dump_path, 'w'))
    
    users = actions[['user_id', 'sku_id']].copy()
    labels = actions['label'].copy()
    del actions['user_id']
    del actions['sku_id']
    del actions['label']

    return users, actions, labels
        

In [38]:
train_start_date = '2016-03-10'
train_end_date = '2016-04-11'
test_start_date = '2016-04-11'
test_end_date = '2016-04-16'


user_index, training_data, label = make_train_set(train_start_date, train_end_date, test_start_date, test_end_date)


In [40]:
training_data.dtypes

2016-04-10-2016-04-11-action_1    float64
2016-04-10-2016-04-11-action_2    float64
2016-04-10-2016-04-11-action_3    float64
2016-04-10-2016-04-11-action_4    float64
2016-04-10-2016-04-11-action_5    float64
2016-04-10-2016-04-11-action_6    float64
2016-04-09-2016-04-11-action_1    float64
2016-04-09-2016-04-11-action_2    float64
2016-04-09-2016-04-11-action_3    float64
2016-04-09-2016-04-11-action_4    float64
2016-04-09-2016-04-11-action_5    float64
2016-04-09-2016-04-11-action_6    float64
2016-04-08-2016-04-11-action_1    float64
2016-04-08-2016-04-11-action_2    float64
2016-04-08-2016-04-11-action_3    float64
2016-04-08-2016-04-11-action_4    float64
2016-04-08-2016-04-11-action_5    float64
2016-04-08-2016-04-11-action_6    float64
2016-04-06-2016-04-11-action_1    float64
2016-04-06-2016-04-11-action_2    float64
2016-04-06-2016-04-11-action_3    float64
2016-04-06-2016-04-11-action_4    float64
2016-04-06-2016-04-11-action_5    float64
2016-04-06-2016-04-11-action_6    

In [32]:
user_index

,user_id,sku_id
0,1,124997
1,17,58959
2,22,15259
3,22,24423
4,22,48154
5,22,48895
6,22,49178
7,22,56234
8,22,75836
9,22,89802


In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(training_data.values, label.values, test_size=0.2, random_state=0)

In [46]:
print X_train.shape
print X_test.shape

(111620, 97)
(27906, 97)


In [47]:

#  make test set
def make_test_set(train_start_date, train_end_date):
    dump_path = './cache/test_set_%s_%s.pkl' % (train_start_date, train_end_date)
    if os.path.exists(dump_path):
        actions = pickle.load(open(dump_path))
    else:
        start_days = "2016-02-01"
        user = get_basic_user_feat()
        product = get_basic_product_feat()
        user_acc = get_accumulate_user_feat(start_days, train_end_date)
        product_acc = get_accumulate_product_feat(start_days, train_end_date)
        comment_acc = get_comments_product_feat(train_start_date, train_end_date)
        #labels = get_labels(test_start_date, test_end_date)

        # generate 时间窗口
        # actions = get_accumulate_action_feat(train_start_date, train_end_date)
        actions = None
        for i in (1, 2, 3, 5, 7, 10, 15, 21, 30):
            start_days = datetime.strptime(train_end_date, '%Y-%m-%d') - timedelta(days=i)
            start_days = start_days.strftime('%Y-%m-%d')
            if actions is None:
                actions = get_action_feat(start_days, train_end_date)
            else:
                actions = pd.merge(actions, get_action_feat(start_days, train_end_date), how='left',
                                   on=['user_id', 'sku_id'])

        actions = pd.merge(actions, user, how='left', on='user_id')
        actions = pd.merge(actions, user_acc, how='left', on='user_id')
        actions = pd.merge(actions, product, how='left', on='sku_id')
        actions = pd.merge(actions, product_acc, how='left', on='sku_id')
        actions = pd.merge(actions, comment_acc, how='left', on='sku_id')
        #actions = pd.merge(actions, labels, how='left', on=['user_id', 'sku_id'])
        actions = actions.fillna(0)
        actions = actions[actions['cate'] == 8]
        pickle.dump(actions, open(dump_path, 'w'))

    users = actions[['user_id', 'sku_id']].copy()
    del actions['user_id']
    del actions['sku_id']
    return users, actions

sub_start_date = '2016-03-15'
sub_end_date = '2016-04-16'
sub_user_index, sub_trainning_data = make_test_set(sub_start_date, sub_end_date,)


comment_date_begin:  2016-04-15


In [48]:
print sub_trainning_data.shape

(70744, 97)


In [50]:
import xgboost as xgb

dtrain=xgb.DMatrix(X_train, label=y_train)
dtest=xgb.DMatrix(X_test, label=y_test)
sub_trainning_data = xgb.DMatrix(sub_trainning_data.values)

/home/zhangguoqiang/genv/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [60]:
# xgboost模型
# 为什么选择xgboost模型，因为树模型对特征处理的要求不高而且效果也相当不错，不管是类别特征，连续特征效果都很好。
# 在Kaggle竞赛中，该模型通常能取得较好成绩。
param = {'learning_rate' : 0.1, 'n_estimators': 1000, 'max_depth': 3, 
        'min_child_weight': 5, 'gamma': 0, 'subsample': 1.0, 'colsample_bytree': 0.8,
        'scale_pos_weight': 1, 'eta': 0.05, 'silent': 1, 'objective': 'binary:logistic'}
num_round = 283
param['nthread'] = 4
#param['eval_metric'] = "auc"
print "param: ", param
plst = param.items()
print "plst: ", plst
plst += [('eval_metric', 'logloss')]
evallist = [(dtest, 'eval'), (dtrain, 'train')]
bst=xgb.train(plst, dtrain, num_round, evallist)

param:  {'scale_pos_weight': 1, 'learning_rate': 0.1, 'n_estimators': 1000, 'colsample_bytree': 0.8, 'silent': 1, 'nthread': 4, 'min_child_weight': 5, 'subsample': 1.0, 'eta': 0.05, 'objective': 'binary:logistic', 'max_depth': 3, 'gamma': 0}
plst:  [('scale_pos_weight', 1), ('learning_rate', 0.1), ('n_estimators', 1000), ('colsample_bytree', 0.8), ('silent', 1), ('nthread', 4), ('min_child_weight', 5), ('subsample', 1.0), ('eta', 0.05), ('objective', 'binary:logistic'), ('max_depth', 3), ('gamma', 0)]
[0]	eval-logloss:0.601106	train-logloss:0.601127
[1]	eval-logloss:0.525733	train-logloss:0.525793
[2]	eval-logloss:0.462907	train-logloss:0.463004
[3]	eval-logloss:0.409887	train-logloss:0.409992
[4]	eval-logloss:0.36463	train-logloss:0.364773
[5]	eval-logloss:0.325711	train-logloss:0.325885
[6]	eval-logloss:0.292021	train-logloss:0.292228
[7]	eval-logloss:0.262693	train-logloss:0.262943
[8]	eval-logloss:0.237052	train-logloss:0.23735
[9]	eval-logloss:0.214558	train-logloss:0.214898
[10]	

In [59]:
help(xgb.train)

Help on function train in module xgboost.training:

train(params, dtrain, num_boost_round=10, evals=(), obj=None, feval=None, maximize=False, early_stopping_rounds=None, evals_result=None, verbose_eval=True, learning_rates=None, xgb_model=None, callbacks=None)
    Train a booster with given parameters.
    
    Parameters
    ----------
    params : dict
        Booster params.
    dtrain : DMatrix
        Data to be trained.
    num_boost_round: int
        Number of boosting iterations.
    evals: list of pairs (DMatrix, string)
        List of items to be evaluated during training, this allows user to watch
        performance on the validation set.
    obj : function
        Customized objective function.
    feval : function
        Customized evaluation function.
    maximize : bool
        Whether to maximize feval.
    early_stopping_rounds: int
        Activates early stopping. Validation error needs to decrease at least
        every <early_stopping_rounds> round(s) to contin

In [71]:
y = bst.predict(sub_trainning_data)
sub_user_index['label'] = y
pred = sub_user_index[sub_user_index['label'] >= 0.03]
pred

,user_id,sku_id,label
13,20,63006,0.048956
1327,1077,57018,0.032789
1721,1467,31662,0.059215
2053,1873,126146,0.034028
2099,1952,131300,0.046648
2190,2060,31662,0.037166
2217,2075,126146,0.068276
2227,2087,149854,0.183384
2276,2139,131300,0.038237
2279,2139,154636,0.053666


In [77]:
pred_res = pred[['user_id', 'sku_id']]
pred_user_sku = pred_res.groupby('user_id').first().reset_index()
pred_user_sku['user_id'] = pred_user_sku['user_id'].astype(int)
pred_user_sku

,user_id,sku_id
0,20,63006
1,1077,57018
2,1467,31662
3,1873,126146
4,1952,131300
5,2060,31662
6,2075,126146
7,2087,149854
8,2139,131300
9,2425,57018
